# Extensions

The wrapper class functionalities can be extended using composition or inheritance.

## Composition

The concept of composition adds a component to a composite class. The relationship is such, that the composite class does not know the component class but wise versa. This allows to add additional functionality without changing the original code. Adding components to an existing wrapper class will be called "registering" here.

In [1]:
import h5rdmtoolbox as h5tbx
h5tbx.use(None)

## Built-in extensions
You may add your own extensions, but the packages comes already with some, that may be useful:

### 1. Vector
The `Vector` extension returns a `xr.Dataset` including the provided datasets when the `Vector` object is sliced. It just saves some time and makes it a one-liner to create the `xr.Dataset`.

In [2]:
from h5rdmtoolbox.extensions import vector

In [18]:
with h5tbx.File() as h5:
    h5.create_dataset('u', data=[1, 2, 3])
    h5.create_dataset('v', data=[2, 2, 2])
    
    vec = h5.Vector(xvel='u', yvel='v')[1:3]
vec

<xarray.Dataset>
Dimensions:  (dim_0: 2)
Dimensions without coordinates: dim_0
Data variables:
    xvel     (dim_0) int32 2 3
    yvel     (dim_0) int32 2 2

### 2. magnitude
This extension let's you compute the magnitude of given `xr.DataArray` objects, which goes well with the above extension:

In [19]:
from h5rdmtoolbox.extensions import magnitude # automatically available when vetor is imported

In [20]:
vec.magnitude.compute_from('xvel', 'yvel')

<xarray.Dataset>
Dimensions:                 (dim_0: 2)
Dimensions without coordinates: dim_0
Data variables:
    xvel                    (dim_0) int32 2 3
    yvel                    (dim_0) int32 2 2
    magnitude_of_xvel_yvel  (dim_0) float64 2.0 3.0

### Registration of standard attributes
During runtime, we can define so-called "standard attributes" and assign them to group or dataset classes. A more detailed explanation can be found [here](../conventions/standard_attributes.ipynb).

In the following example, the standard attribute shall write user information to attributes. For this the name and the orcid is expected. The new standard attribute shall be assigned to the `Dataset` class of the current wrapper:

In [ ]:
from h5rdmtoolbox.conventions import StandardAttribute

In [ ]:
class UserName(StandardAttribute):
    def set(self, username):
        # username shall be something like ['Max', '0000-1111-2222-3333']
        assert len(username) == 2
        super().set(name='user_name', value=user_name_data[0])
        super().set(name='user_orcid', value=user_name_data[1])
        
    def get(self):
        return super().get('user_name'), super().get('user_orcid')

Let's check if it worked out:

In [ ]:
with h5tbx.File() as h5:
    ds = h5.create_dataset('test', shape=(2,))    
    ds.username = 'First User', '0000-0001-8729-0482'
    print(ds.username)
    try:
        ds.username = 'Second User'
    except ValueError as e :
        print(f'Could not add user due to: {e}')
    print(ds.username)
    h5.dump()

### Registration of datasets
We can also "register" dataset accessors. In the following example we add "device" as a "property with methods". So "device" seems to be a property which has a method "add". Such an implementation faclitates the interaction with HDF data, too. Note, that this "property-like" accessor is available for all `Dataset` objects from know on in this session:

In [ ]:
from h5rdmtoolbox.wrapper.accessory import register_special_dataset
@register_special_dataset('device', h5tbx.Dataset, overwrite=True)
class DeviceProperty:
    """Device Accessor class"""

    def __init__(self, ds):
        self._ds = ds
        self._device_name = 'NoDeviceName'
        
    def add(self, new_device_name):
        """adds the attribute device_name to the dataset"""
        self._ds.attrs['device_name'] = new_device_name
        
    @property
    def name(self):
        return self._ds.attrs['device_name']

In [ ]:
with h5tbx.File() as h5:
    ds = h5.create_dataset('test', shape=(2,))
    print(type(ds))
    ds.device.add('my device')
    print(ds.device.name)